In [1]:
#This notebook

In [2]:
#import libraries
import pandas as pd
import yfinance as yf
import requests
import time
import json
import os
import datetime
from quickfs import QuickFS
from dotenv import load_dotenv
from Quantapp.EconomicData import EconomicData
qe = EconomicData()
load_dotenv()
#from edgar import *
#set_identity("Joey blum joeyblum@indigo.com")



False

In [3]:
#parameters
should_update = False

In [4]:
#create client to query quickfs
#set api key for quickfs
api_key = '72418c68701409373b2902e70c58096c0c9d827e'
os.environ['API_QUICKFS'] = api_key

#create quickfs client
api_key = os.environ['API_QUICKFS']
client = QuickFS(api_key)


In [5]:
from IPython.display import display
import pandas as pd

#set parameters

should_update = False #careful with this one, it will update all the data and use up you credits. set to True if you want to force an update on the data

sp500_companies = qe.retrieve_market_tables()['SP500_TABLE']
#Sectors: ['Industrials', 'Health Care', 'Information Technology', 'Utilities', 'Financials', 'Materials', 'Consumer Discretionary', 'Real Estate', 'Communication Services', 'Consumer Staples', 'Energy']
#add a query column to the dataframe
sp500_companies['Query'] = sp500_companies['Symbol'] + ':US'
#columns are 'Symbol', 'Sector', 'Sub-Industry' 

Information_Technology_companies = sp500_companies[sp500_companies['Sector'] == 'Information Technology']
Health_Care_companies            = sp500_companies[sp500_companies['Sector'] == 'Health Care']
Consumer_Discretionary_companies = sp500_companies[sp500_companies['Sector'] == 'Consumer Discretionary']
Utilities_companies              = sp500_companies[sp500_companies['Sector'] == 'Utilities']
Financials_companies             = sp500_companies[sp500_companies['Sector'] == 'Financials']
Materials_companies              = sp500_companies[sp500_companies['Sector'] == 'Materials']
Real_Estate_companies            = sp500_companies[sp500_companies['Sector'] == 'Real Estate']
Consumer_Staples_companies       = sp500_companies[sp500_companies['Sector'] == 'Consumer Staples']
Industrials_companies            = sp500_companies[sp500_companies['Sector'] == 'Industrials']
Energy_companies                 = sp500_companies[sp500_companies['Sector'] == 'Energy']
Communication_Services_companies = sp500_companies[sp500_companies['Sector'] == 'Communication Services']


# Create a dictionary mapping each sector to its companies DataFrame
sectors = {
    'Information Technology': Information_Technology_companies,
    'Health Care': Health_Care_companies,
    'Consumer Discretionary': Consumer_Discretionary_companies,
    'Utilities': Utilities_companies,
    'Financials': Financials_companies,
    'Materials': Materials_companies,
    'Real Estate': Real_Estate_companies,
    'Consumer Staples': Consumer_Staples_companies,
    'Industrials': Industrials_companies,
    'Energy': Energy_companies,
    'Communication Services': Communication_Services_companies
}




In [6]:
# Neatly display each sector and its unique Sub-Industries as a DataFrame
import plotly.graph_objects as go

# Build a dictionary mapping each sector to its list of unique sub-industries
sector_subs = {}
for sector, df in sectors.items():
    sector_subs[sector] = list(df['Sub-Industry'].unique())

# Determine the maximum number of sub-industries across all sectors
max_len = max(len(lst) for lst in sector_subs.values())

# Pad each list with empty strings so that all lists have the same length
for key, lst in sector_subs.items():
    if len(lst) < max_len:
        sector_subs[key] = lst + [''] * (max_len - len(lst))

# Create a DataFrame where each column is a sector and each row a sub-industry (or empty)
table_df = pd.DataFrame(sector_subs)

# Create a Plotly table with sectors as columns and sub-industries as rows
fig = go.Figure(data=[go.Table(
    header=dict(
        values=list(table_df.columns),
        fill_color='paleturquoise',
        align='left'
    ),
    cells=dict(
        values=[table_df[col] for col in table_df.columns],
        fill_color='lavender',
        align='left'
    )
)])

fig.update_layout(height=1000)
fig.show()

In [7]:
#define custom functions

#convinence functions for working with dates
#------------------------------------------------------------
def simplify_datetime_index(series):
    """
    Simplifies the DateTime index of a Series to contain only the date (YYYY-MM-DD),
    maintaining it as a DateTimeIndex without timezone information.
    
    Parameters:
        series (pd.Series): The input Series with a DateTimeIndex.
    
    Returns:
        pd.Series: The Series with the DateTime index simplified to YYYY-MM-DD.
    """
    if not isinstance(series.index, pd.DatetimeIndex):
        raise TypeError("The Series index must be a DateTimeIndex.")
    
    # Remove timezone information if present
    if series.index.tz is not None:
        series = series.copy()
        series.index = series.index.tz_convert('UTC').tz_localize(None)
    
    # Normalize the index to remove the time component
    series.index = series.index.normalize()
    
    return series
#flattens date from YYYY-MM-DD to YYYYMMDD
def flatten_date(date_str):
    date_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d")
    return date_obj.strftime("%Y%m%d")
#------------------------------------------------------------





#Call to API
#------------------------------------------------------------
#function to get all available metrics in QuickFS
def get_metrics(client):
    """
    Retrieves the metrics available from the QuickFS API.
    
    Parameters:
        client (QuickFSClient): The client object used to interact with the API.
    
    Returns:
        list: A list of dictionaries containing the available metrics and their descriptions.
    """
    # Retrieve the metrics available from the API
    metrics = client.get_available_metrics()
    
    # Return the list of metrics
    return metrics

def get_latest_earnings_date(ticker):
    earnings_dates = yf.Ticker(ticker).earnings_dates.dropna().index
    last_earnings_date = str(earnings_dates[0].strftime('%Y-%m-%d'))
    return last_earnings_date

def get_companies_updated_on_or_after_date(client, date):
    flatten_date = flatten_date(date)
    updated_companies = client.get_updated_companies(country='US', date=flatten_date)
    return updated_companies

def retrieve_data_from_API(ticker_string,client):
    print('Querying data API!')
    #convert dictionary to 
    data = client.get_data_full(symbol=ticker_string)
    data = process_full_data(data, client)
    print('Data retreived successfully!\n')
    save_company_info(ticker_string, data['financials_annual'], data['financials_quarterly'], data['metadata'])
    print('Data saved successfully!\n')
    return data

def retrieve_data(ticker_string, data_type='annual', statement_type='income_statement', should_update=False, client=client):
    # Check if the ticker folder exists; if not, create it
    parent_folder_name = 'company_data'
    ticker_folder = os.path.join(parent_folder_name, ticker_string)
    if not os.path.exists(ticker_folder):
        os.makedirs(ticker_folder)
        print('Company folder does not exist, creating it now!\n')
    else:
        print('Company folder exists\n')

    # Check if the company folder is empty
    if len(os.listdir(ticker_folder)) == 0:
        print(ticker_folder)
        print('No data found for this company\n')
        retrieve_data_from_API(ticker_string, client)
    else:
        print('Data found for this company\n')
        
    if should_update:
        print('Manually updating data\n')
        retrieve_data_from_API(ticker_string, client)
    else:
        print('Set should_update to True to manually update data\n')
        
    # Check if the data is up to date; if not, retrieve it from the API
    # (This comment is a reminder for future implementation if needed.)

    # If the data exists and is up to date, load it from disk according to the fiscal period
    if data_type == 'annual':
        print('Retrieving annual data\n')
        return pd.read_csv(os.path.join(ticker_folder, data_type, statement_type + '.csv'))
    elif data_type == 'quarterly':
        print('Retrieving quarterly data\n')
        return pd.read_csv(os.path.join(ticker_folder, data_type, statement_type + '.csv'))
    elif data_type == 'metadata':
        print('Retrieving metadata\n')
        with open(os.path.join(ticker_folder, 'metadata.json')) as f:
            return json.load(f)
    else:
        print('Invalid fiscal period\n')

def retrieve_data_for_companies(companies, data_type='annual', statement_type='income_statement', should_update=False, client=client):
    data = {}
    for company in companies:
        ticker = company.split(':')[0]
        print(f'Retrieving data for {ticker}')
        data[ticker] = retrieve_data(ticker, data_type, statement_type, should_update, client)
    
    #print usage of data
    print(client.get_usage())
    return data
        
#------------------------------------------------------------



#Procesing data once it is retreived

#------------------------------------------------------------
def process_full_data(full_data, client):
    """
    Processes the full_data dictionary to extract company metadata and financials.
    It converts financial data (annual and quarterly) into DataFrames with sorted columns,
    and then separates them into dictionaries based on statement types using available metrics.
    
    Parameters:
        full_data (dict): Dictionary containing keys 'metadata' and 'financials' (with keys 'annual' and 'quarterly').
        client: Parameter used to retrieve metrics via get_metrics(client).
    
    Returns:
        dict: A dictionary containing:
            'ticker': company ticker,
            'metadata': company metadata,
            'financials_annual': dictionary of annual financial DataFrames,
            'financials_quarterly': dictionary of quarterly financial DataFrames.
    """
    import pandas as pd

    def create_financials_dict(financials_df, client):
        # Retrieve the metrics from the client using get_metrics
        
        metrics = get_metrics(client)
        # Identify metrics that are not present in the DataFrame
        metrics_not_in_df = []
        for field in metrics:
            if field['metric'] not in financials_df.columns:
                metrics_not_in_df.append(field['metric'])

        # Filter out metrics not in the DataFrame and sort the remaining metrics
        metrics = [field for field in metrics if field['metric'] not in metrics_not_in_df]
        metrics = sorted(metrics, key=lambda x: x['metric'])
        

        
        
        # Extract metrics based on statement type
        metrics_income_statement    = [field['metric'] for field in metrics if field['statement_type'] == 'income_statement']
        metrics_balance_sheet       = [field['metric'] for field in metrics if field['statement_type'] == 'balance_sheet']
        metrics_cash_flow_statement = [field['metric'] for field in metrics if field['statement_type'] == 'cash_flow_statement']
        metrics_computed             = [field['metric'] for field in metrics if field['statement_type'] == 'computed']
        metrics_misc                = [field['metric'] for field in metrics if field['statement_type'] == 'misc']
  
        # Create separate DataFrames for each statement type by concatenating metrics_misc with the respective group
        income_statement_df     = pd.concat([financials_df[metrics_misc], financials_df[metrics_income_statement]], axis=1)
        balance_sheet_df        = pd.concat([financials_df[metrics_misc], financials_df[metrics_balance_sheet]], axis=1)
        cash_flow_statement_df  = pd.concat([financials_df[metrics_misc], financials_df[metrics_cash_flow_statement]], axis=1)
        computed_df              = pd.concat([financials_df[metrics_misc], financials_df[metrics_computed]], axis=1)
        misc_df                 = financials_df[metrics_misc]
        
        #fiscal_year_key
        #fiscal_year_number
        #fiscal_quarter_key
        #fiscal_quarter_number 
        
        #check if fiscal_year_key, fiscal_year_number, fiscal_quarter_key, fiscal_quarter_number are in the financials_df
        #if they are, add columns to each dataframe
        
        if 'fiscal_year_key' in financials_df.columns:
            income_statement_df['fiscal_year_key'] = financials_df['fiscal_year_key']
            balance_sheet_df['fiscal_year_key'] = financials_df['fiscal_year_key']
            cash_flow_statement_df['fiscal_year_key'] = financials_df['fiscal_year_key']
            computed_df['fiscal_year_key'] = financials_df['fiscal_year_key']
            misc_df['fiscal_year_key'] = financials_df['fiscal_year_key']
            
        if 'fiscal_year_number' in financials_df.columns:
            income_statement_df['fiscal_year_number'] = financials_df['fiscal_year_number']
            balance_sheet_df['fiscal_year_number'] = financials_df['fiscal_year_number']
            cash_flow_statement_df['fiscal_year_number'] = financials_df['fiscal_year_number']
            computed_df['fiscal_year_number'] = financials_df['fiscal_year_number']
            misc_df['fiscal_year_number'] = financials_df['fiscal_year_number']
            
        if 'fiscal_quarter_key' in financials_df.columns:
            income_statement_df['fiscal_quarter_key'] = financials_df['fiscal_quarter_key']
            balance_sheet_df['fiscal_quarter_key'] = financials_df['fiscal_quarter_key']
            cash_flow_statement_df['fiscal_quarter_key'] = financials_df['fiscal_quarter_key']
            computed_df['fiscal_quarter_key'] = financials_df['fiscal_quarter_key']
            misc_df['fiscal_quarter_key'] = financials_df['fiscal_quarter_key']
            
        if 'fiscal_quarter_number' in financials_df.columns:
            income_statement_df['fiscal_quarter_number'] = financials_df['fiscal_quarter_number']
            balance_sheet_df['fiscal_quarter_number'] = financials_df['fiscal_quarter_number']
            cash_flow_statement_df['fiscal_quarter_number'] = financials_df['fiscal_quarter_number']
            computed_df['fiscal_quarter_number'] = financials_df['fiscal_quarter_number']
            misc_df['fiscal_quarter_number'] = financials_df['fiscal_quarter_number']
             
        # Return a dictionary with the separated DataFrames
        return {
            'income_statement': income_statement_df,
            'balance_sheet': balance_sheet_df,
            'cash_flow_statement': cash_flow_statement_df,
            'computed': computed_df,
            'misc': misc_df
        }
    
    # Get ticker; default to 'AAPL' if not provided in full_data
    ticker = full_data['metadata']['symbol']
    
    # Extract metadata and financials from full_data
    company_metadata     = full_data['metadata']
    financials_annual    = full_data['financials']['annual']
    financials_quarterly = full_data['financials']['quarterly']
    
    # Create DataFrames for annual and quarterly financials with sorted columns
    financials_annual_df = pd.DataFrame(financials_annual).reindex(
        sorted(pd.DataFrame(financials_annual).columns), axis=1
    )
    financials_quarterly_df = pd.DataFrame(financials_quarterly).reindex(
        sorted(pd.DataFrame(financials_quarterly).columns), axis=1
    )
    

    
    # Create dictionaries for annual and quarterly financials
    financials_annual_dict    = create_financials_dict(financials_annual_df, client)
    financials_quarterly_dict = create_financials_dict(financials_quarterly_df, client)
    
    return {
        'ticker': ticker,
        'metadata': company_metadata,
        'financials_annual': financials_annual_dict,
        'financials_quarterly': financials_quarterly_dict
    }

def save_company_info(ticker, financials_data_annual, financials_data_quarterly, company_metadata):
    """
    Saves company financial data (annual and quarterly) and metadata to disk.
    
    This function:
      - Creates a folder for the given ticker under 'company_data' if it doesn't exist.
      - For each period ('annual' and 'quarterly'), creates a subfolder and saves each
        financial DataFrame as a CSV file.
      - Saves the company metadata as a JSON file in the ticker folder.
    
    Parameters:
      ticker (str): The company ticker.
      financials_data_annual (dict): Dictionary of annual financial DataFrames.
      financials_data_quarterly (dict): Dictionary of quarterly financial DataFrames.
      company_metadata (dict): Dictionary containing company metadata.
    """
    import os, json

    parent_folder = 'company_data'
    ticker_folder = os.path.join(parent_folder, ticker)
    if not os.path.exists(ticker_folder):
        os.makedirs(ticker_folder)
        print(f"Created folder: {ticker_folder}")

    # Function to save financial data for a given period
    def save_financials_for_period(financials_data, period):
        period_folder = os.path.join(ticker_folder, period)
        if not os.path.exists(period_folder):
            os.makedirs(period_folder)
            print(f"Created folder for {period} data: {period_folder}")

        for key, df in financials_data.items():
            file_path = os.path.join(period_folder, f'{key}.csv')
            df.to_csv(file_path)
            print(f"{key} data saved to {file_path}")

    # Save annual financials
    save_financials_for_period(financials_data_annual, 'annual')

    # Save quarterly financials
    save_financials_for_period(financials_data_quarterly, 'quarterly')

    # Save company metadata to JSON
    metadata_path = os.path.join(ticker_folder, 'metadata.json')
    with open(metadata_path, 'w') as f:
        json.dump(company_metadata, f, indent=4)
    print(f"Metadata saved to {metadata_path}")

#update the log file with the latest data
def update_log(ticker_str, last_updated, latest_earnings_date, is_updated_on_server):
    parent_folder_name = 'company_data'
    
    #if log file does not exist, create it using create_log
    if not os.path.exists(os.path.join(parent_folder_name, 'log.csv')):
        create_log()
        
    #retreive the log file as a dataframe, 
    #check if the ticker exists in the log file, if it does, update the row with the new data
    #if it does not, create a new row with the new data
    log_file_path = os.path.join(parent_folder_name, 'log.csv')
    log_df = pd.read_csv(log_file_path)
    
    if ticker_str in log_df['ticker_name'].values:
        log_df.loc[log_df['ticker_name'] == ticker_str, 'last_updated'] = last_updated
        log_df.loc[log_df['ticker_name'] == ticker_str, 'latest_earnings_date'] = latest_earnings_date
        log_df.loc[log_df['ticker_name'] == ticker_str, 'is_updated_on_server'] = is_updated_on_server
    else:
        new_entry = pd.DataFrame({'ticker_name': [ticker_str], 
                                'last_updated': [last_updated], 
                                'latest_earnings_date': [latest_earnings_date], 
                                'is_updated_on_server': [is_updated_on_server]})
        log_df = pd.concat([log_df, new_entry], ignore_index=True)
    #sort the log by ticker name
    log_df = log_df.sort_values(by='ticker_name')
    log_df.to_csv(log_file_path, index=False)
    print('Log updated')

 #creates an empty log file if it does not exist

def create_log():
    parent_folder_name = 'company_data'
    # if log file does not exist, create it, it should be a csv
    # else print that it exists
    log_file_path = os.path.join(parent_folder_name, 'log.csv')
    column_names = ['ticker_name', 'last_updated', 'latest_earnings_date','is_updated_on_server']
    column_names_str = ','.join(column_names)
    
    if not os.path.exists(log_file_path):
        print('Log File doesnt exist! creating it now')
        with open(log_file_path, 'w') as f:
            f.write(column_names_str + '\n')
    else:
        print('Log File already exists')    

def get_metric_df_from_companies(companies, metric_key, data_type='quarterly', 
                                 statement_type='computed', should_update=False):
    """
    Retrieves a DataFrame for a given metric (column from data) across multiple companies.

    Parameters:
        companies (list): List of company identifiers in the format 'TICKER:COUNTRY'
        metric_key (str): The key in the data dict to extract (e.g., 'revenue_growth', 'price_to_earnings')
        data_type (str): Data frequency, default is 'quarterly'
        statement_type (str): Type of statement to retrieve, default is 'computed'
        should_update (bool): Flag whether to force update data

    Returns:
        pd.DataFrame: DataFrame with period end dates as index and each column containing the metric for a company.
    """
    import pandas as pd
    # Initialize an empty DataFrame to collect series
    metric_df = pd.DataFrame()

    for company in companies:
        # Extract ticker from the string, assumed to be in the format 'TICKER:COUNTRY'
        company_ = company.split(':')[0]
        data = retrieve_data(company_, data_type=data_type, statement_type=statement_type, 
                               should_update=should_update)
        
        # Get the period labels and the desired metric values
        if data_type == 'annual':
            index_dates = data['fiscal_year_key']  # e.g., ['FY2002', 'FY2003', ...]
        else:
            index_dates = data['fiscal_quarter_key']  # e.g., ['FY2002.FQ1', 'FY2002.FQ2', ...]
        metric_series = data[metric_key]            # e.g., [12.34, 23.45, ..., 34.56, 45.67]
        print('company:', company_)
        print(metric_series)
        
        # Set the index of the metric series to the period labels and rename it to the company ticker
        metric_series.index = index_dates
        metric_series = metric_series.rename(company_)
        
        # Concatenate the new Series to the DataFrame aligning by the index (period labels)
        metric_df = pd.concat([metric_df, metric_series], axis=1)

    # Optionally, sort the resulting DataFrame by the index
    metric_df.sort_index(inplace=True)
    return metric_df

#------------------------------------------------------------





#plot

In [8]:
#Information for queries
usage_data = client.get_usage()['quota']
print("Data Usage:")
print(f"  Used     : {usage_data['used']}")
print(f"  Remaining: {usage_data['remaining']}")
print(f"  Resets   : {usage_data['resets']}")

income_statment_fields = retrieve_data('AAPL',statement_type='income_statement' ,should_update=should_update).columns.to_list()
balance_sheet_fields = retrieve_data('AAPL',statement_type='balance_sheet' ,should_update=should_update).columns.to_list()
cash_flow_statement_fields = retrieve_data('AAPL',statement_type='cash_flow_statement' ,should_update=should_update).columns.to_list()
computed_fields = retrieve_data('AAPL',statement_type='computed' ,should_update=should_update).columns.to_list()
misc_fields = retrieve_data('AAPL',statement_type='misc' ,should_update=should_update).columns.to_list()

#neatly print the fields
print('Income Statment Fields:')
display(income_statment_fields)
print('\n')
print('Balance Sheet Fields:')
display(balance_sheet_fields)
print('\n')
print('Cash Flow Statement Fields:')
display(cash_flow_statement_fields)
print('\n')
print('Computed Fields:')
display(computed_fields)
print('\n')
print('Misc Fields:')
display(misc_fields)
print('\n')



Data Usage:
  Used     : 0
  Remaining: 500
  Resets   : 2025-03-24T04:00:00Z
Company folder exists

Data found for this company

Set should_update to True to manually update data

Retrieving annual data

Company folder exists

Data found for this company

Set should_update to True to manually update data

Retrieving annual data

Company folder exists

Data found for this company

Set should_update to True to manually update data

Retrieving annual data

Company folder exists

Data found for this company

Set should_update to True to manually update data

Retrieving annual data

Company folder exists

Data found for this company

Set should_update to True to manually update data

Retrieving annual data

Income Statment Fields:


['Unnamed: 0',
 'dividends',
 'original_filing_date',
 'period_end_date',
 'period_end_price',
 'shares_eop',
 'cogs',
 'da_income_statement_supplemental',
 'eps_basic',
 'eps_diluted',
 'gross_profit',
 'income_allocated_to_minority_interest',
 'income_tax',
 'interest_expense',
 'interest_income',
 'net_income',
 'net_income_available_to_shareholders',
 'net_income_continuing',
 'net_income_discontinued',
 'net_interest_income_normal',
 'operating_income',
 'other_income_statement_items',
 'other_nonoperating_income',
 'other_opex',
 'preferred_dividends',
 'pretax_income',
 'revenue',
 'rnd',
 'sga',
 'shares_basic',
 'shares_diluted',
 'special_charges',
 'total_opex',
 'fiscal_year_key',
 'fiscal_year_number']



Balance Sheet Fields:


['Unnamed: 0',
 'dividends',
 'original_filing_date',
 'period_end_date',
 'period_end_price',
 'shares_eop',
 'accounts_payable',
 'accumulated_depreciation',
 'aoci',
 'apic',
 'cash_and_equiv',
 'common_stock',
 'current_accrued_liabilities',
 'current_capital_leases',
 'current_deferred_revenue',
 'current_deferred_tax_liability',
 'equity_and_other_investments',
 'goodwill',
 'intangible_assets',
 'inventories',
 'lt_debt',
 'minority_interest_liability',
 'noncurrent_capital_leases',
 'noncurrent_deferred_revenue',
 'other_current_assets',
 'other_current_liabilities',
 'other_equity',
 'other_lt_assets',
 'other_lt_liabilities',
 'pension_liabilities',
 'ppe_gross',
 'ppe_net',
 'preferred_stock',
 'receivables',
 'retained_earnings',
 'st_debt',
 'st_investments',
 'tax_payable',
 'total_assets',
 'total_current_assets',
 'total_current_liabilities',
 'total_equity',
 'total_liabilities',
 'total_liabilities_and_equity',
 'total_noncurrent_assets',
 'total_noncurrent_liabilitie



Cash Flow Statement Fields:


['Unnamed: 0',
 'dividends',
 'original_filing_date',
 'period_end_date',
 'period_end_price',
 'shares_eop',
 'capex',
 'cf_cff',
 'cf_cfi',
 'cf_cfo',
 'cf_forex',
 'cf_net_change_in_cash',
 'cff_common_stock_issued',
 'cff_common_stock_net',
 'cff_common_stock_repurchased',
 'cff_debt_issued',
 'cff_debt_net',
 'cff_debt_repaid',
 'cff_dividend_paid',
 'cff_other',
 'cff_pfd_issued',
 'cff_pfd_net',
 'cff_pfd_repurchased',
 'cfi_acquisitions',
 'cfi_acquisitions_net',
 'cfi_divestitures',
 'cfi_intangibles_net',
 'cfi_investment_net',
 'cfi_investment_purchases',
 'cfi_investment_sales',
 'cfi_other',
 'cfi_ppe_net',
 'cfi_ppe_purchases',
 'cfi_ppe_sales',
 'cfo_change_in_working_capital',
 'cfo_da',
 'cfo_deferred_tax',
 'cfo_inventory',
 'cfo_net_income',
 'cfo_other_noncash_items',
 'cfo_other_working_capital',
 'cfo_prepaid_expenses',
 'cfo_receivables',
 'cfo_stock_comp',
 'fiscal_year_key',
 'fiscal_year_number']



Computed Fields:


['Unnamed: 0',
 'dividends',
 'original_filing_date',
 'period_end_date',
 'period_end_price',
 'shares_eop',
 'assets_to_equity',
 'assets_to_equity_median',
 'book_value',
 'book_value_per_share',
 'capex_growth',
 'cash_and_equiv_growth',
 'cf_cfo_cagr_10',
 'cfo_growth',
 'current_ratio',
 'debt_to_assets',
 'debt_to_assets_median',
 'debt_to_equity',
 'debt_to_equity_median',
 'dividends_per_share_cagr_10',
 'dividends_per_share_growth',
 'ebitda',
 'ebitda_growth',
 'ebitda_margin',
 'ebitda_per_share',
 'enterprise_value',
 'enterprise_value_to_book',
 'enterprise_value_to_earnings',
 'enterprise_value_to_fcf',
 'enterprise_value_to_pretax_income',
 'enterprise_value_to_sales',
 'enterprise_value_to_tangible_book',
 'eps_diluted_cagr_10',
 'eps_diluted_growth',
 'equity_to_assets',
 'fcf',
 'fcf_cagr_10',
 'fcf_growth',
 'fcf_margin',
 'fcf_margin_median',
 'fcf_per_share',
 'gross_margin',
 'gross_margin_median',
 'gross_profit_growth',
 'income_tax_rate',
 'market_cap',
 'net_



Misc Fields:


['Unnamed: 0',
 'dividends',
 'original_filing_date',
 'period_end_date',
 'period_end_price',
 'shares_eop',
 'fiscal_year_key',
 'fiscal_year_number']

In [9]:


Revenues = {
    'Information Technology': get_metric_df_from_companies(Information_Technology_companies['Query'].to_list(), 'revenue', statement_type='income_statement', should_update=should_update),
    'Health Care': get_metric_df_from_companies(Health_Care_companies['Query'].to_list(), 'revenue', statement_type='income_statement', should_update=should_update),
    #'Consumer Discretionary': get_metric_df_from_companies(Consumer_Discretionary_companies['Query'].to_list(), 'revenue', statement_type='income_statement', should_update=should_update),
    'Utilities': get_metric_df_from_companies(Utilities_companies['Query'].to_list(), 'revenue', statement_type='income_statement', should_update=should_update),
    'Financials': get_metric_df_from_companies(Financials_companies['Query'].to_list(), 'revenue', statement_type='income_statement', should_update=should_update),
    'Materials': get_metric_df_from_companies(Materials_companies['Query'].to_list(), 'revenue', statement_type='income_statement', should_update=should_update),
    'Real Estate': get_metric_df_from_companies(Real_Estate_companies['Query'].to_list(), 'revenue', statement_type='income_statement', should_update=should_update),
    'Consumer Staples': get_metric_df_from_companies(Consumer_Staples_companies['Query'].to_list(), 'revenue', statement_type='income_statement', should_update=should_update),
    'Industrials': get_metric_df_from_companies(Industrials_companies['Query'].to_list(), 'revenue', statement_type='income_statement', should_update=should_update),
    'Energy': get_metric_df_from_companies(Energy_companies['Query'].to_list(), 'revenue', statement_type='income_statement', should_update=should_update)
}

Operating_Income = {
    'Information Technology': get_metric_df_from_companies(Information_Technology_companies['Query'].to_list(), 'operating_income', statement_type='income_statement', should_update=should_update),
    'Health Care': get_metric_df_from_companies(Health_Care_companies['Query'].to_list(), 'operating_income', statement_type='income_statement', should_update=should_update),
   # 'Consumer Discretionary': get_metric_df_from_companies(Consumer_Discretionary_companies['Query'].to_list(), 'operating_income', statement_type='income_statement', should_update=should_update),
    'Utilities': get_metric_df_from_companies(Utilities_companies['Query'].to_list(), 'operating_income', statement_type='income_statement', should_update=should_update),
  #  'Financials': get_metric_df_from_companies(Financials_companies['Query'].to_list(), 'operating_income', statement_type='income_statement', should_update=should_update),
    'Materials': get_metric_df_from_companies(Materials_companies['Query'].to_list(), 'operating_income', statement_type='income_statement', should_update=should_update),
    'Real Estate': get_metric_df_from_companies(Real_Estate_companies['Query'].to_list(), 'operating_income', statement_type='income_statement', should_update=should_update),
    'Consumer Staples': get_metric_df_from_companies(Consumer_Staples_companies['Query'].to_list(), 'operating_income', statement_type='income_statement', should_update=should_update),
    'Industrials': get_metric_df_from_companies(Industrials_companies['Query'].to_list(), 'operating_income', statement_type='income_statement', should_update=should_update),
    'Energy': get_metric_df_from_companies(Energy_companies['Query'].to_list(), 'operating_income', statement_type='income_statement', should_update=should_update)
}

Net_Income = {
    'Information Technology': get_metric_df_from_companies(Information_Technology_companies['Query'].to_list(), 'net_income', statement_type='income_statement', should_update=should_update),
    'Health Care': get_metric_df_from_companies(Health_Care_companies['Query'].to_list(), 'net_income', statement_type='income_statement', should_update=should_update),
    'Consumer Discretionary': get_metric_df_from_companies(Consumer_Discretionary_companies['Query'].to_list(), 'net_income', statement_type='income_statement', should_update=should_update),
    'Utilities': get_metric_df_from_companies(Utilities_companies['Query'].to_list(), 'net_income', statement_type='income_statement', should_update=should_update),
    'Financials': get_metric_df_from_companies(Financials_companies['Query'].to_list(), 'net_income', statement_type='income_statement', should_update=should_update),
    'Materials': get_metric_df_from_companies(Materials_companies['Query'].to_list(), 'net_income', statement_type='income_statement', should_update=should_update),
    'Real Estate': get_metric_df_from_companies(Real_Estate_companies['Query'].to_list(), 'net_income', statement_type='income_statement', should_update=should_update),
    'Consumer Staples': get_metric_df_from_companies(Consumer_Staples_companies['Query'].to_list(), 'net_income', statement_type='income_statement', should_update=should_update),
    'Industrials': get_metric_df_from_companies(Industrials_companies['Query'].to_list(), 'net_income', statement_type='income_statement', should_update=should_update),
    'Energy': get_metric_df_from_companies(Energy_companies['Query'].to_list(), 'net_income', statement_type='income_statement', should_update=should_update)
}



Gross_Margin = {
    'Information Technology': get_metric_df_from_companies(Information_Technology_companies['Query'].to_list(), 'gross_margin', statement_type='computed', should_update=should_update),
   # 'Health Care': get_metric_df_from_companies(Health_Care_companies['Query'].to_list(), 'gross_margin', statement_type='computed', should_update=should_update),
    'Consumer Discretionary': get_metric_df_from_companies(Consumer_Discretionary_companies['Query'].to_list(), 'gross_margin', statement_type='computed', should_update=should_update),
    'Utilities': get_metric_df_from_companies(Utilities_companies['Query'].to_list(), 'gross_margin', statement_type='computed', should_update=should_update),
   # 'Financials': get_metric_df_from_companies(Financials_companies['Query'].to_list(), 'gross_margin', statement_type='computed', should_update=should_update),
    'Materials': get_metric_df_from_companies(Materials_companies['Query'].to_list(), 'gross_margin', statement_type='computed', should_update=should_update),
    'Real Estate': get_metric_df_from_companies(Real_Estate_companies['Query'].to_list(), 'gross_margin', statement_type='computed', should_update=should_update),
    'Consumer Staples': get_metric_df_from_companies(Consumer_Staples_companies['Query'].to_list(), 'gross_margin', statement_type='computed', should_update=should_update),
    'Industrials': get_metric_df_from_companies(Industrials_companies['Query'].to_list(), 'gross_margin', statement_type='computed', should_update=should_update),
    'Energy': get_metric_df_from_companies(Energy_companies['Query'].to_list(), 'gross_margin', statement_type='computed', should_update=should_update)
}

Operating_Margin = {
    'Information Technology': get_metric_df_from_companies(Information_Technology_companies['Query'].to_list(), 'operating_margin', statement_type='computed', should_update=should_update),
    'Health Care': get_metric_df_from_companies(Health_Care_companies['Query'].to_list(), 'operating_margin', statement_type='computed', should_update=should_update),
    'Consumer Discretionary': get_metric_df_from_companies(Consumer_Discretionary_companies['Query'].to_list(), 'operating_margin', statement_type='computed', should_update=should_update),
    'Utilities': get_metric_df_from_companies(Utilities_companies['Query'].to_list(), 'operating_margin', statement_type='computed', should_update=should_update),
    #'Financials': get_metric_df_from_companies(Financials_companies['Query'].to_list(), 'operating_margin', statement_type='computed', should_update=should_update),
    'Materials': get_metric_df_from_companies(Materials_companies['Query'].to_list(), 'operating_margin', statement_type='computed', should_update=should_update),
    'Real Estate': get_metric_df_from_companies(Real_Estate_companies['Query'].to_list(), 'operating_margin', statement_type='computed', should_update=should_update),
    'Consumer Staples': get_metric_df_from_companies(Consumer_Staples_companies['Query'].to_list(), 'operating_margin', statement_type='computed', should_update=should_update),
    'Industrials': get_metric_df_from_companies(Industrials_companies['Query'].to_list(), 'operating_margin', statement_type='computed', should_update=should_update),
    'Energy': get_metric_df_from_companies(Energy_companies['Query'].to_list(), 'operating_margin', statement_type='computed', should_update=should_update)
}

Net_Margin = {
    'Information Technology': get_metric_df_from_companies(Information_Technology_companies['Query'].to_list(), 'net_income_margin', statement_type='computed', should_update=should_update),
    'Health Care': get_metric_df_from_companies(Health_Care_companies['Query'].to_list(), 'net_income_margin', statement_type='computed', should_update=should_update),
    'Consumer Discretionary': get_metric_df_from_companies(Consumer_Discretionary_companies['Query'].to_list(), 'net_income_margin', statement_type='computed', should_update=should_update),
    'Utilities': get_metric_df_from_companies(Utilities_companies['Query'].to_list(), 'net_income_margin', statement_type='computed', should_update=should_update),
    'Financials': get_metric_df_from_companies(Financials_companies['Query'].to_list(), 'net_income_margin', statement_type='computed', should_update=should_update),
    'Materials': get_metric_df_from_companies(Materials_companies['Query'].to_list(), 'net_income_margin', statement_type='computed', should_update=should_update),
    'Real Estate': get_metric_df_from_companies(Real_Estate_companies['Query'].to_list(), 'net_income_margin', statement_type='computed', should_update=should_update),
    'Consumer Staples': get_metric_df_from_companies(Consumer_Staples_companies['Query'].to_list(), 'net_income_margin', statement_type='computed', should_update=should_update),
    'Industrials': get_metric_df_from_companies(Industrials_companies['Query'].to_list(), 'net_income_margin', statement_type='computed', should_update=should_update),
    'Energy': get_metric_df_from_companies(Energy_companies['Query'].to_list(), 'net_income_margin', statement_type='computed', should_update=should_update)
}




Revenues_mean = pd.DataFrame({
    'Information Technology': Revenues['Information Technology'].fillna(method='ffill').mean(axis=1),
    'Health Care': Revenues['Health Care'].fillna(method='ffill').mean(axis=1),
    #'Consumer Discretionary': Revenues['Consumer Discretionary'].fillna(method='ffill').mean(axis=1),
    'Utilities': Revenues['Utilities'].fillna(method='ffill').mean(axis=1),
    'Financials': Revenues['Financials'].fillna(method='ffill').mean(axis=1),
    'Materials': Revenues['Materials'].fillna(method='ffill').mean(axis=1),
    'Real Estate': Revenues['Real Estate'].fillna(method='ffill').mean(axis=1),
    'Consumer Staples': Revenues['Consumer Staples'].fillna(method='ffill').mean(axis=1),
    'Industrials': Revenues['Industrials'].fillna(method='ffill').mean(axis=1),
    'Energy': Revenues['Energy'].fillna(method='ffill').mean(axis=1)
})

Operating_Income_mean = pd.DataFrame({
    'Information Technology': Operating_Income['Information Technology'].fillna(method='ffill').mean(axis=1),
    'Health Care': Operating_Income['Health Care'].fillna(method='ffill').mean(axis=1),
   # 'Consumer Discretionary': Operating_Income['Consumer Discretionary'].fillna(method='ffill').mean(axis=1),
    'Utilities': Operating_Income['Utilities'].fillna(method='ffill').mean(axis=1),
    #'Financials': Operating_Income['Financials'].fillna(method='ffill').mean(axis=1),
    'Materials': Operating_Income['Materials'].fillna(method='ffill').mean(axis=1),
    'Real Estate': Operating_Income['Real Estate'].fillna(method='ffill').mean(axis=1),
    'Consumer Staples': Operating_Income['Consumer Staples'].fillna(method='ffill').mean(axis=1),
    'Industrials': Operating_Income['Industrials'].fillna(method='ffill').mean(axis=1),
    'Energy': Operating_Income['Energy'].fillna(method='ffill').mean(axis=1)
})

Net_Income_mean = pd.DataFrame({
    'Information Technology': Net_Income['Information Technology'].fillna(method='ffill').mean(axis=1),
    'Health Care': Net_Income['Health Care'].fillna(method='ffill').mean(axis=1),
    'Consumer Discretionary': Net_Income['Consumer Discretionary'].fillna(method='ffill').mean(axis=1),
    'Utilities': Net_Income['Utilities'].fillna(method='ffill').mean(axis=1),
    'Financials': Net_Income['Financials'].fillna(method='ffill').mean(axis=1),
    'Materials': Net_Income['Materials'].fillna(method='ffill').mean(axis=1),
    'Real Estate': Net_Income['Real Estate'].fillna(method='ffill').mean(axis=1),
    'Consumer Staples': Net_Income['Consumer Staples'].fillna(method='ffill').mean(axis=1),
    'Industrials': Net_Income['Industrials'].fillna(method='ffill').mean(axis=1),
    'Energy': Net_Income['Energy'].fillna(method='ffill').mean(axis=1)
})


Revenue_growth = {
    'Information Technology': get_metric_df_from_companies(Information_Technology_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Health Care': get_metric_df_from_companies(Health_Care_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Consumer Discretionary': get_metric_df_from_companies(Consumer_Discretionary_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Utilities': get_metric_df_from_companies(Utilities_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Financials': get_metric_df_from_companies(Financials_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Materials': get_metric_df_from_companies(Materials_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Real Estate': get_metric_df_from_companies(Real_Estate_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Consumer Staples': get_metric_df_from_companies(Consumer_Staples_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Industrials': get_metric_df_from_companies(Industrials_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Energy': get_metric_df_from_companies(Energy_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update)
}




Gross_Margin_median = pd.DataFrame({
    'Information Technology': Gross_Margin['Information Technology'].fillna(method='ffill').median(axis=1),
    #'Health Care': Gross_Margin['Health Care'].fillna(method='ffill').median(axis=1),
    'Consumer Discretionary': Gross_Margin['Consumer Discretionary'].fillna(method='ffill').median(axis=1),
    'Utilities': Gross_Margin['Utilities'].fillna(method='ffill').median(axis=1),
    #'Financials': Gross_Margin['Financials'].fillna(method='ffill').median(axis=1),
    'Materials': Gross_Margin['Materials'].fillna(method='ffill').median(axis=1),
    'Real Estate': Gross_Margin['Real Estate'].fillna(method='ffill').median(axis=1),
    'Consumer Staples': Gross_Margin['Consumer Staples'].fillna(method='ffill').median(axis=1),
    'Industrials': Gross_Margin['Industrials'].fillna(method='ffill').median(axis=1),
    'Energy': Gross_Margin['Energy'].fillna(method='ffill').median(axis=1)
})

Operating_Margin_median = pd.DataFrame({
    'Information Technology': Operating_Margin['Information Technology'].fillna(method='ffill').median(axis=1),
    'Health Care': Operating_Margin['Health Care'].fillna(method='ffill').median(axis=1),
    'Consumer Discretionary': Operating_Margin['Consumer Discretionary'].fillna(method='ffill').median(axis=1),
    'Utilities': Operating_Margin['Utilities'].fillna(method='ffill').median(axis=1),
    #'Financials': Operating_Margin['Financials'].fillna(method='ffill').median(axis=1),
    'Materials': Operating_Margin['Materials'].fillna(method='ffill').median(axis=1),
    'Real Estate': Operating_Margin['Real Estate'].fillna(method='ffill').median(axis=1),
    'Consumer Staples': Operating_Margin['Consumer Staples'].fillna(method='ffill').median(axis=1),
    'Industrials': Operating_Margin['Industrials'].fillna(method='ffill').median(axis=1),
    'Energy': Operating_Margin['Energy'].fillna(method='ffill').median(axis=1)
})

Net_Margin_median = pd.DataFrame({
    'Information Technology': Net_Margin['Information Technology'].fillna(method='ffill').median(axis=1),
    'Health Care': Net_Margin['Health Care'].fillna(method='ffill').median(axis=1),
    'Consumer Discretionary': Net_Margin['Consumer Discretionary'].fillna(method='ffill').median(axis=1),
    'Utilities': Net_Margin['Utilities'].fillna(method='ffill').median(axis=1),
    'Financials': Net_Margin['Financials'].fillna(method='ffill').median(axis=1),
    'Materials': Net_Margin['Materials'].fillna(method='ffill').median(axis=1),
    'Real Estate': Net_Margin['Real Estate'].fillna(method='ffill').median(axis=1),
    'Consumer Staples': Net_Margin['Consumer Staples'].fillna(method='ffill').median(axis=1),
    'Industrials': Net_Margin['Industrials'].fillna(method='ffill').median(axis=1),
    'Energy': Net_Margin['Energy'].fillna(method='ffill').median(axis=1)
})





Company folder exists

Data found for this company

Set should_update to True to manually update data

Retrieving quarterly data

company: ACN
0      3327447000
1      3189023000
2      3418429000
3      3462341000
4      3574488000
         ...     
84    16224303000
85    15799514000
86    16466828000
87    16405819000
88    17689545000
Name: revenue, Length: 89, dtype: int64
Company folder exists

Data found for this company

Set should_update to True to manually update data

Retrieving quarterly data

company: ADBE
0      267896000
1      317359000
2      284879000
3      294654000
4      296895000
         ...    
87    5048000000
88    5182000000
89    5309000000
90    5408000000
91    5606000000
Name: revenue, Length: 92, dtype: int64
Company folder exists

Data found for this company

Set should_update to True to manually update data

Retrieving quarterly data

company: AMD
0      902073000
1      600299000
2      508227000
3      686430000
4      714555000
         ...    
87 

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\1419935105.py:85: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\1419935105.py:86: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\1419935105.py:88: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\1419935105.py:89: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\1419935105.py:90: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() 

company: JNPR
0    -0.6319
1    -0.4211
2    -0.2462
3     0.0280
4     0.2862
       ...  
87   -0.0579
88   -0.1624
89   -0.1681
90   -0.0477
91    0.0287
Name: revenue_growth, Length: 92, dtype: float64
Company folder exists

Data found for this company

Set should_update to True to manually update data

Retrieving quarterly data

company: KEYS
0     0.0000
1     0.0000
2     0.0526
3     0.0000
4     0.0000
5    -0.0706
6    -0.0223
7     0.0798
8     0.0808
9     0.0447
10   -0.0040
11   -0.1215
12   -0.0157
13    0.0285
14   -0.0121
15    0.0751
16    0.0013
17    0.0069
18    0.0300
19    0.1636
20    0.1691
21    0.1528
22    0.3147
23    0.2067
24    0.1924
25    0.2019
26    0.1010
27    0.0826
28    0.0697
29    0.0884
30   -0.1788
31   -0.0699
32    0.0892
33    0.0776
34    0.3642
35    0.2324
36    0.0606
37    0.0593
38    0.1064
39    0.1043
40    0.1151
41    0.1048
42    0.0288
43    0.0043
44   -0.0914
45   -0.0883
46   -0.1251
47   -0.1193
48   -0.0183
Name: revenue

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\1419935105.py:141: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\1419935105.py:143: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\1419935105.py:144: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\1419935105.py:146: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\1419935105.py:147: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffi

In [10]:

fig = go.Figure()
for sector in Revenues_mean.columns:
    fig.add_trace(go.Scatter(x=Revenues_mean.index, y=Revenues_mean[sector], mode='lines', name=sector))
fig.update_layout(title='Mean of Revenues by Sector', xaxis_title='Date', yaxis_title='Revenue', height=800)
fig.show()

fig = go.Figure()
for sector in Operating_Income_mean.columns:
    fig.add_trace(go.Scatter(x=Operating_Income_mean.index, y=Operating_Income_mean[sector], mode='lines', name=sector))
fig.update_layout(title='Mean of Operating Income by Sector', xaxis_title='Date', yaxis_title='Operating Income', height=800)
fig.show()

fig = go.Figure()
for sector in Net_Income_mean.columns:
    fig.add_trace(go.Scatter(x=Net_Income_mean.index, y=Net_Income_mean[sector], mode='lines', name=sector))
fig.update_layout(title='Mean of Net Income by Sector', xaxis_title='Date', yaxis_title='Net Income', height=800)
fig.show()



In [11]:
fig = go.Figure()
for sector in Gross_Margin_median.columns:
    fig.add_trace(go.Scatter(x=Gross_Margin_median.index, y=Gross_Margin_median[sector], mode='lines', name=sector))
fig.add_shape(
    type="line",
    xref="paper", x0=0, x1=1,  # spans the whole x-axis
    yref="y", y0=0, y1=0,
    line=dict(color="black", width=1, dash="dash")
)
fig.update_layout(title='Median of Gross Margin by Sector', xaxis_title='Date', yaxis_title='Gross Margin', height=800)
fig.show()

fig = go.Figure()
for sector in Operating_Margin_median.columns:
    fig.add_trace(go.Scatter(x=Operating_Margin_median.index, y=Operating_Margin_median[sector], mode='lines', name=sector))
fig.add_shape(
    type="line",
    xref="paper", x0=0, x1=1,
    yref="y", y0=0, y1=0,
    line=dict(color="black", width=1, dash="dash")
)
fig.update_layout(title='Median of Operating Margin by Sector', xaxis_title='Date', yaxis_title='Operating Margin', height=800)
fig.show()

fig = go.Figure()
for sector in Net_Margin_median.columns:
    fig.add_trace(go.Scatter(x=Net_Margin_median.index, y=Net_Margin_median[sector], mode='lines', name=sector))
fig.add_shape(
    type="line",
    xref="paper", x0=0, x1=1,
    yref="y", y0=0, y1=0,
    line=dict(color="black", width=1, dash="dash")
)
fig.update_layout(title='Median of Net Margin by Sector', xaxis_title='Date', yaxis_title='Net Margin', height=800)
fig.show()



In [12]:
Revenue_growth = {
    'Information Technology': get_metric_df_from_companies(Information_Technology_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Health Care': get_metric_df_from_companies(Health_Care_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Consumer Discretionary': get_metric_df_from_companies(Consumer_Discretionary_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Utilities': get_metric_df_from_companies(Utilities_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Financials': get_metric_df_from_companies(Financials_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Materials': get_metric_df_from_companies(Materials_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Real Estate': get_metric_df_from_companies(Real_Estate_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Consumer Staples': get_metric_df_from_companies(Consumer_Staples_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Industrials': get_metric_df_from_companies(Industrials_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update),
    'Energy': get_metric_df_from_companies(Energy_companies['Query'].to_list(), 'revenue_growth', statement_type='computed', should_update=should_update)
}

Revenue_growth_mean = pd.DataFrame({
    'Information Technology': Revenue_growth['Information Technology'].fillna(method='ffill').mean(axis=1),
    'Health Care': Revenue_growth['Health Care'].fillna(method='ffill').mean(axis=1),
    'Consumer Discretionary': Revenue_growth['Consumer Discretionary'].fillna(method='ffill').mean(axis=1),
    'Utilities': Revenue_growth['Utilities'].fillna(method='ffill').mean(axis=1),
    'Financials': Revenue_growth['Financials'].fillna(method='ffill').mean(axis=1),
    'Materials': Revenue_growth['Materials'].fillna(method='ffill').mean(axis=1),
    'Real Estate': Revenue_growth['Real Estate'].fillna(method='ffill').mean(axis=1),
    'Consumer Staples': Revenue_growth['Consumer Staples'].fillna(method='ffill').mean(axis=1),
    'Industrials': Revenue_growth['Industrials'].fillna(method='ffill').mean(axis=1),
    'Energy': Revenue_growth['Energy'].fillna(method='ffill').mean(axis=1)
})

fig = go.Figure()
for sector in Revenue_growth_mean.columns:
    fig.add_trace(go.Scatter(x=Revenue_growth_mean.index, y=Revenue_growth_mean[sector], mode='lines', name=sector))
fig.add_shape(
    type="line",
    xref="paper", x0=0, x1=1,
    yref="y", y0=0, y1=0,
    line=dict(color="black", width=1, dash="dash")
)
fig.update_layout(title='Mean of Revenue Growth by Sector', xaxis_title='Date', yaxis_title='Revenue Growth', height=800)
fig.show()


Company folder exists

Data found for this company

Set should_update to True to manually update data

Retrieving quarterly data

company: ACN
0    -0.0041
1    -0.0418
2     0.0202
3     0.1224
4     0.0742
       ...  
84    0.0302
85   -0.0009
86   -0.0059
87    0.0263
88    0.0903
Name: revenue_growth, Length: 89, dtype: float64
Company folder exists

Data found for this company

Set should_update to True to manually update data

Retrieving quarterly data

company: ADBE
0    -0.1856
1    -0.0776
2    -0.0247
3     0.1138
4     0.1082
       ...  
87    0.1155
88    0.1132
89    0.1023
90    0.1059
91    0.1105
Name: revenue_growth, Length: 92, dtype: float64
Company folder exists

Data found for this company

Set should_update to True to manually update data

Retrieving quarterly data

company: AMD
0    -0.2411
1    -0.3907
2    -0.3364
3    -0.2788
4    -0.2078
       ...  
87    0.1016
88    0.0224
89    0.0888
90    0.1756
91    0.2415
Name: revenue_growth, Length: 92, dtype: fl

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\727060257.py:15: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\727060257.py:16: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\727060257.py:17: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\727060257.py:18: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\kevin\AppData\Local\Temp\ipykernel_5420\727060257.py:19: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or ob